# Reformatter and resampler of APPA's data
mainly by *Samuele Facenda*

copy-paste contributed by *Magoni Parker*


--- 

convert the table with a column for every channel to a table(with a table with the chenges of sensing materials) with a feature that is the sensing material, age of the sensor, sin and cos of date and a lot of other things


In [2]:
import math
import locale
import numpy as np
import pandas as pd

from tqdm import tqdm
from numpy import NaN


NUMBER_OF_SENSORS = 8

In [3]:
def get_sensing_mat(index_date, channel):
    global timeline
    return timeline[dates_str[index_date]][timeline.channel == channel].values[0]

def find_change_date(index_date_reach, channel):
    global timeline, dates_str, dates, NUMBER_OF_SENSORS

    actual_sens_mat = get_sensing_mat(index_date_reach, channel)
    if  not '*' in actual_sens_mat:
        return dates[index_date_reach]
    else:
        # find the last change
        if get_sensing_mat(index_date_reach - 1,channel) in actual_sens_mat:
            # do the same thing on the previus cell
            return find_change_date(index_date_reach - 1, channel)
        else:
            # the sensor changed channel, find the channel and do the same thing
            channel_before = 1
            while channel_before <= NUMBER_OF_SENSORS:
                if get_sensing_mat(index_date_reach -1,channel_before) in actual_sens_mat:
                    break
                channel_before += 1
            else:
                raise Exception(f"error in find previus position of sensor\n actual index {index_date_reach} type {actual_sens_mat}")
            return find_change_date(index_date_reach - 1, channel_before)


In [4]:
def elaborate_dataset(dataset, out_file):
    columns = ['ts','sensing_material','signal_res','heater_res','heater_volt','T','RH','P','sin_hour','cos_hour','sin_ordate','cos_ordate','sin_weekday','cos_weekday','year','age']
    new_data = pd.DataFrame(columns=columns)

    last_change_date = [dates[0] for _ in range(NUMBER_OF_SENSORS)]
    index_date_reached = 0
    actual_sensing_material = [get_sensing_mat(0, i) for i in range(1, NUMBER_OF_SENSORS + 1)]

    for index, record in tqdm(dataset.iterrows(), total = dataset.shape[0]):
        if record.ts.date() < dates[0].date():
            # skip all data before sensor changing(and sensing mat knowing)
            continue

        if index_date_reached + 1 != len(dates) and record.ts.date() == dates[index_date_reached + 1].date():
            # reached sensing material change day
            index_date_reached += 1
            for channel in range(NUMBER_OF_SENSORS):
                last_change_date[channel] = find_change_date(index_date_reached, channel+1)

            # update the sensing material array
            actual_sensing_material = [get_sensing_mat(index_date_reached,i) for i in range(1, NUMBER_OF_SENSORS + 1)]

        append_ds = pd.DataFrame(columns=columns)
        for channel in range(1, NUMBER_OF_SENSORS + 1):
            if pd.isnull(record["Rs" + str(channel)]):
                continue
            
            # for every channel add a row
            append_ds.loc[len(append_ds.index)] = [
                record['ts'],
                actual_sensing_material[channel - 1].replace("*",""),
                record['Rs' + str(channel)],
                record['Rh' + str(channel)],
                record['Vh' + str(channel)],
                record['T'],
                record['RH'],
                record['P'],
                math.sin(record['ts'].hour) * 2 * (math.pi / 24),
                math.cos(record['ts'].hour) * 2 * (math.pi / 24),
                math.sin(record['ts'].toordinal()) * 2 * (math.pi / 366),
                math.cos(record['ts'].toordinal()) * 2 * (math.pi / 366),
                math.sin(record['ts'].weekday()) * 2 * (math.pi / 7),
                math.cos(record['ts'].weekday()) * 2 * (math.pi / 7),
                record['ts'].year,
                (record['ts'] - last_change_date[channel - 1]).days
            ]
            
        new_data = pd.concat(objs=(new_data, append_ds))
    new_data = new_data.dropna().reset_index(drop=True)
    new_data.to_csv(f'{out_file}.csv', index=False)

In [5]:
def create_dataset(df_file, timeline_file):
    global timeline, NUMBER_OF_SECONDS

    df = pd.read_csv(f'{df_file}.csv')
    timeline = pd.read_csv(f'{timeline_file}.csv')

    # get the timestamp of the sensing material change date
    global dates_str, dates
    dates_str = timeline.columns.to_list()
    dates_str.remove("channel")
    dates = [pd.to_datetime(x) for x in dates_str]
    appa_1 = df[df.ts.str.contains("\0") == False]
    
    # change raw_table ts column to timestamp
    df.ts = pd.to_datetime(df.ts.str.slice(start = 0, stop = 24), format = "%a %b %d %Y %H:%M:%S")
    
    # set and reset index to computate the mea
    df = df.set_index("ts")
    df = df.resample("H").mean()
    df = df.reset_index(level=0)

    elaborate_dataset(df, f'{df_file}_adjusted')

In [6]:
create_dataset('appa1', 'appa1_timeline')
create_dataset('appa2', 'appa2_timeline')

100%|██████████| 25517/25517 [05:17<00:00, 80.35it/s]   
